In [ ]:

import warnings,os,uuid
# remove warnings from ax
os.environ["PYTHONWARNINGS"] = "ignore"
warnings.filterwarnings(action='ignore')
import ax, torch
from optimpv import *
from optimpv.axBOtorch.axUtils import *
import pandas as pd
import numpy as np
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch, copy, os
from itertools import combinations
from ax import *
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import init_notebook_plotting, render
from botorch.models import SaasFullyBayesianSingleTaskGP, SingleTaskGP

init_notebook_plotting()

# define the parameters
params = []

mun = FitParam(name = 'l1.mu_n', value = 2e-8, bounds = [1e-8,1e-7], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-8, rescale = True, stepsize = None, display_name=r'$\mu_n$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mun)

mup = FitParam(name = 'l1.mu_p', value = 8e-8, bounds = [1e-8,1e-7], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-8, rescale = True, stepsize = None, display_name=r'$\mu_p$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mup)

k_direct = FitParam(name = 'l1.k_direct', value = 1e-17, bounds = [1e-18,1e-16], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = True, stepsize = None, display_name=r'$k_{direct}$', unit='m$^3$ s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(k_direct)

Nc = FitParam(name = 'l1.N_c', value = 1e27, bounds = [5e26,5e27], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_c$', unit='m$^{-3}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(Nc)


# original values
params_orig = copy.deepcopy(params)



In [ ]:
import pySIMsalabim as sim
from pySIMsalabim.experiments.JV_steady_state import *

curr_dir = os.getcwd()
session_path = os.path.join(curr_dir, 'SIMsalabim','SimSS')
simss_device_parameters = os.path.join(session_path, 'simulation_setup.txt')

# Set the JV parameters
Gfracs = [0.1,0.5,1.0] # Fractions of the generation rate to simulate
# Gfracs = None
UUID = str(uuid.uuid4())

cmd_pars = []
for param in params:
    cmd_pars.append({'par':param.name, 'val':str(param.value)})


# Run the JV simulation
ret, mess = run_SS_JV(simss_device_parameters, session_path, JV_file_name = 'JV.dat', varFile= 'Var.dat',G_fracs = Gfracs, parallel = False, max_jobs = 3, UUID=UUID, cmd_pars=cmd_pars)

# import random noise
from numpy.random import default_rng
# save data for fitting
X,y = [],[]
if Gfracs is None:
    data = pd.read_csv(os.path.join(session_path, 'JV_'+UUID+'.dat'), sep=r'\s+') # Load the data
    Vext = np.asarray(data['Vext'].values)
    Jext = np.asarray(data['Jext'].values)
    G = np.ones_like(Vext)
    rng = default_rng()#
    noise = rng.standard_normal(Jext.shape) * 0.01 * Jext
    Jext = Jext + noise
    X= Vext
    y = Jext

    plt.figure()
    plt.plot(X,y)
    plt.show()
else:
    for Gfrac in Gfracs:
        data = pd.read_csv(os.path.join(session_path, 'JV_Gfrac_'+str(Gfrac)+'_'+UUID+'.dat'), sep=r'\s+') # Load the data
        Vext = np.asarray(data['Vext'].values)
        Jext = np.asarray(data['Jext'].values)
        G = np.ones_like(Vext)*Gfrac
        rng = default_rng()#
        noise = rng.standard_normal(Jext.shape) * 0.005 * Jext
        Jext = Jext + noise

        if len(X) == 0:
            X = np.vstack((Vext,G)).T
            y = Jext
        else:
            X = np.vstack((X,np.vstack((Vext,G)).T))
            y = np.hstack((y,Jext))

    # remove all the current where Jext is positive i.e. above Voc
    X = X[y<100]
    y = y[y<100]

    plt.figure()
    for Gfrac in Gfracs:
        plt.plot(X[X[:,1]==Gfrac,0],y[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac))
    plt.legend()
    plt.show()




In [3]:
from optimpv.DDfits.JVAgent import JVAgent
metric = 'mse'
loss = 'soft_l1'

jv = JVAgent(params, X, y, session_path, simss_device_parameters, parallel = False, max_jobs = 3, metric = metric, loss = loss)

In [4]:
from optimpv.axBOtorch.axBOtorchOptimizer import axBOtorchOptimizer
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from ax.modelbridge.transforms.standardize_y import StandardizeY
from ax.modelbridge.transforms.unit_x import UnitX
from ax.modelbridge.transforms.remove_fixed import RemoveFixed
from ax.modelbridge.transforms.log import Log

model_gen_kwargs_list = None
parameter_constraints = None

model_kwargs_list = [{},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY]}]
# n_batches = [1,140], batch_size = [40,4]                     
optimizer = axBOtorchOptimizer(params = params, agents = jv, models = ['SOBOL','BOTORCH_MODULAR'],n_batches = [10,30], batch_size = [4,2], ax_client = None,  max_parallelism = 100,
                   model_kwargs_list = model_kwargs_list, model_gen_kwargs_list = model_gen_kwargs_list, name = 'ax_opti',parameter_constraints = parameter_constraints,scheduler_logging_level = 0)


In [ ]:
optimizer.optimize(True)

In [ ]:
ax_client = optimizer.ax_client
best_parameters = ax_client.get_best_parameters()[0]
print(best_parameters)
jv.params_w(best_parameters,jv.params)
print(jv.params)
print(jv.get_SIMsalabim_clean_cmd(jv.params))
jv.package_SIMsalabim_files(jv.params,'simss')

In [ ]:
# plot the evolution of the optimization
render(ax_client.get_contour_plot(param_x="l1.mu_n", param_y="l1.mu_p", metric_name=optimizer.all_metrics[0]))

from ax.plot.slice import plot_slice
model = ax_client.generation_strategy.model

render(plot_slice(model=model, param_name="l1.k_direct", metric_name=optimizer.all_metrics[0]))




In [ ]:
data = ax_client.experiment.fetch_data()

plt.plot(np.minimum.accumulate(data.df["mean"]), label="Best value seen so far")



In [ ]:
# get name of all parameters that are not 'fixed'
from optimpv.posterior.posterior import *

params_orig_dict = {}
for idx, p in enumerate(params_orig):
    if p.value_type == 'float':
        if p.force_log:
            best_parameters[p.name] = 10**(best_parameters[p.name])
        else:
            best_parameters[p.name] = best_parameters[p.name]*p.fscale
    elif p.value_type == 'int':
        best_parameters[p.name] = best_parameters[p.name]*p.stepsize
    else:
        best_parameters[p.name] = best_parameters[p.name]

    params_orig_dict[p.name] = p.value


fig_dens, ax_dens = plot_density_exploration(params, optimizer.all_metrics[0], optimizer = optimizer, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')


In [ ]:
# rerun the simulation with the best parameters
yfit = jv.run(parameters=ax_client.get_best_parameters()[0])
# print(jv.run_Ax(parameters=results.get_best_result(metric=metric,mode='min',filter_nan_and_inf=True).config))
plt.figure(figsize=(10,10))
for Gfrac in Gfracs:
    plt.plot(X[X[:,1]==Gfrac,0],y[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac))
    plt.plot(X[X[:,1]==Gfrac,0],yfit[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac)+' fit',linestyle='--')
plt.legend()
plt.show()

In [ ]:
from ax.modelbridge.cross_validation import cross_validate
from ax.plot.contour import interact_contour
from ax.plot.diagnostic import interact_cross_validation
cv_results = cross_validate(model)
render(interact_cross_validation(cv_results))

In [ ]:
# flush cuda memory
Nres = 10
objective_name = optimizer.all_metrics[0]
model = optimizer.ax_client.generation_strategy.model
# set 
    
fig, ax = devils_plot(params, Nres, objective_name, model, loss, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')

In [ ]:
fig, ax = plot_1d_posteriors(params, Nres, objective_name, model, loss, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')

In [ ]:
fig, ax = plot_1D_2D_posterior(params, 'l1.mu_n', 'l1.mu_p', 10, objective_name, model, loss, best_parameters = best_parameters, params_orig = params_orig_dict, optimizer_type = 'ax')

In [ ]:
from optimpv.general.general import inv_loss_function
df = get_df_from_ax(params, optimizer.all_metrics[0], optimizer = optimizer)

# make a second df with the 10 best results
df_best = df.nsmallest(10, objective_name)
# all objectivevalues below 3
# df_best = df[df[objective_name] < 3]

import matplotlib.pyplot as plt
import matplotlib
plt.figure(figsize=(10, 6))

lognorm = matplotlib.colors.LogNorm(vmin=df[objective_name].min(), vmax=df[objective_name].max())

# params_name = ['l2.N_t_bulk', 'l2.mu_p']
# params_name = ['l2.N_t_bulk', 'l1.N_t_int']
# params_name = ['l2.N_t_bulk', 'l2.N_ions']
params_name = ['l1.mu_n', 'l1.mu_p']
sc = plt.scatter(df[params_name[0]], df[params_name[1]], c=df[objective_name], norm=lognorm, cmap='viridis_r')
plt.scatter(df_best[params_name[0]], df_best[params_name[1]], c='grey', marker='o', label='10 best results')
# plot best result
plt.scatter(best_parameters[params_name[0]], best_parameters[params_name[1]], c='blue', marker='x', label='Best result')
# plot initial points
plt.scatter(params_orig_dict[params_name[0]], params_orig_dict[params_name[1]], c='red', marker='x', label='Initial values')
plt.yscale('log')
plt.xscale('log')
plt.colorbar(sc, label='JV_JV_mse')
plt.xlabel(params_name[0])
plt.ylabel(params_name[1])
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Make and density plot of df_best 1d

for i in range(len(params)):
    plt.figure()
    
    # hist with log scale y bins
    logbins = np.geomspace(params[i].bounds[0], params[i].bounds[1], 100)
    plt.hist(df_best[params[i].name], bins=logbins, alpha=0.5, label='10 best results', color='b')
    # kde plt
    sns.kdeplot(df_best[params[i].name], log_scale=True, label='10 best results', color='b')

    plt.vlines(best_parameters[params[i].name], 0, 5, colors='r', linestyles='dashed', label='Best result')
    plt.vlines(params_orig_dict[params[i].name], 0, 5, colors='k', linestyles='dashed', label='Initial values')
    # add median mean std and IQR to plot as vlines and shaded area
    plt.vlines(df_best[params[i].name].median(), 0, 5, colors='g', linestyles='dashed', label='Median')
    plt.vlines(df_best[params[i].name].mean(), 0, 5, colors='orange', linestyles='dashed', label='Mean')
    # plt.vlines(df_best[params[i].name].mean() + df_best[params[i].name].std(), 0, 3, colors='purple', linestyles='dashed', label='Mean + std')
    # plt.vlines(df_best[params[i].name].mean() - df_best[params[i].name].std(), 0, 3, colors='purple', linestyles='dashed', label='Mean - std')
    plt.fill_betweenx([0, 5], df_best[params[i].name].mean() - df_best[params[i].name].std(), df_best[params[i].name].mean() + df_best[params[i].name].std(), color='purple', alpha=0.3, label='Mean +/- std')
    # plt.fill_betweenx([0, 5], df_best[params[i].name].quantile(0.25), df_best[params[i].name].quantile(0.75), color='g', alpha=0.3, label='IQR')

    plt.legend()
    plt.xscale('log')
    plt.xlim(params[i].bounds[0], params[i].bounds[1])
plt.show()


In [16]:
# Clean up the output files (comment out if you want to keep the output files)
sim.clean_all_output(session_path)
sim.delete_folders('tmp',session_path)